In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [3]:
# read in csv file
all_cases = gpd.read_file('/Users/leahwallihan/Durham_school_planning/Res_Development')

In [4]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]

In [5]:
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID','DEN','DISAP','EXP'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [6]:
keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' housing', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
pattern = '|'.join(keywords)
filtered1 = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]

#use not pattern to avoid these words
keywords_avoid = ['addition','expand','storage', 'zoning', 'grading']
pattern_avoid = '|'.join(keywords_avoid)
filtered2 = filtered1[~filtered1['A_DESCRIPT'].str.contains(pattern_avoid, case=False, na=False)]

In [7]:
#filter for all status dates after 2020
filtered2['A_STATUS_D'] = pd.to_datetime(filtered2['A_STATUS_D'])
filtered3 = filtered2[filtered2['A_STATUS_D'].dt.year>=2020]
# filtered3

/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
# function to extract unit quantity from description
def extract_units(description):
    description = re.sub(pattern=r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq. ft.|sqft)', 
                         repl='', string=description, flags=re.IGNORECASE) # remove square footage descriptors

    match_pattern = r'(\d{1,3})(?:\s+[A-Za-z-]+){0,3}?\s+(units?|lots?|homes?|houses?|townhomes?|townhouses?|town homes?|town houses?|apartments?|duplex(?:es)?)'
    matches = re.findall(match_pattern, description, flags=re.IGNORECASE) # match residence types with their quantity

    return matches # returns a list of tuples in format (quantity, residence type)

In [9]:
filtered3['match_results'] = filtered3['A_DESCRIPT'].apply(extract_units)
filtered3.head()

/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,AppType,CasePlanne,EMAIL,ORIG_FID,CreationDa,Creator,EditDate,Editor,geometry,match_results
90,D2000291,PL_MINSP,2020-12-02,APP,2021-09-03,Umstead Grove Conservation Subdivision,"50 Single - family lots, 1 stormwater pond, ad...",JESSICADO,COURTNEYMC,APP,...,Level 4 - Administrative Site Plan,Courtney McQueen,Courtney.McQueen@durhamnc.gov,91,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2016072.97 846210.373),"[(50, lots)]"
224,D1800378,PL_MINSP,2018-11-28,APP,2020-04-14,Elan Innovation District,Mixe of uses including retail and residential....,JOHNRA,TREYFI,APP,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,225,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2028333.906 818879.684),[]
242,D2100156,PL_MINSP,2021-06-04,APP,2022-02-02,ALTA Rutherford,Multifamily Apartments,KIMRO,TREYFI,APP,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,243,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2020729.037 822564.644),[]
265,D1900171,PL_MINSP,2019-05-30,APP,2020-04-03,Ellis Road Phase 3,"37 Townhome units, 102 Attached S-F units (cal...",ROBINSH,COLERE,APP,...,Level 4 - Administrative Site Plan,Cole Renigar,Cole.Renigar@durhamnc.gov,266,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2040942.168 801418.946),"[(37, Townhome), (102, units), (146, lots)]"
290,A1900003,PL_CPAA,2019-02-14,COM,2020-08-17,Cole Property,"From Rural Density Residential (RDR, 0.5DU/acr...",DCULTRA,EMILYST,COM,...,Comprehensive Plan Map or Plan Amendment,Emily Struthers,Emily.Struthers@durhamnc.gov,291,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2065915.279 815129.529),[]


,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,AppType,CasePlanne,EMAIL,ORIG_FID,CreationDa,Creator,EditDate,Editor,geometry,match_results
90,D2000291,PL_MINSP,2020-12-02,APP,2021-09-03,Umstead Grove Conservation Subdivision,"50 Single - family lots, 1 stormwater pond, ad...",JESSICADO,COURTNEYMC,APP,...,Level 4 - Administrative Site Plan,Courtney McQueen,Courtney.McQueen@durhamnc.gov,91,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2016072.97 846210.373),"[(50, lots)]"
224,D1800378,PL_MINSP,2018-11-28,APP,2020-04-14,Elan Innovation District,Mixe of uses including retail and residential....,JOHNRA,TREYFI,APP,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,225,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2028333.906 818879.684),[]
242,D2100156,PL_MINSP,2021-06-04,APP,2022-02-02,ALTA Rutherford,Multifamily Apartments,KIMRO,TREYFI,APP,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,243,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2020729.037 822564.644),[]
265,D1900171,PL_MINSP,2019-05-30,APP,2020-04-03,Ellis Road Phase 3,"37 Townhome units, 102 Attached S-F units (cal...",ROBINSH,COLERE,APP,...,Level 4 - Administrative Site Plan,Cole Renigar,Cole.Renigar@durhamnc.gov,266,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2040942.168 801418.946),"[(37, Townhome), (102, units), (146, lots)]"
290,A1900003,PL_CPAA,2019-02-14,COM,2020-08-17,Cole Property,"From Rural Density Residential (RDR, 0.5DU/acr...",DCULTRA,EMILYST,COM,...,Comprehensive Plan Map or Plan Amendment,Emily Struthers,Emily.Struthers@durhamnc.gov,291,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2065915.279 815129.529),[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21082,D2500050,PL_MINSP,2025-02-28,UN_RE,2025-05-08,803 Sherron Road Townhomes,New townhome development with 53 units and new...,FRANCISCOM,COLERE,UN_RE,...,Level 4 - Administrative Site Plan,Cole Renigar,Cole.Renigar@durhamnc.gov,21083,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2057018.502 805288.203),"[(53, units)]"
21141,D2500072,PL_MINSP,2025-03-18,UN_RE,2025-03-18,Leigh Valley,Apartment development to include 324 units.,MICHAELMA,TREYFI,UN_RE,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21142,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2002541.241 789409.246),"[(324, units)]"
21176,D2500088,PL_MINSP,2025-04-01,UN_RE,2025-04-01,Cornwallis Townhomes,Create 33 townhome site.,EARLENETHO,JALISAHA,UN_RE,...,Level 4 - Administrative Site Plan,JALISA HARRIS,Jalisa.Harris@durhamnc.gov,21177,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2031643.791 796227.263),"[(33, townhome)]"
21241,D2500103,PL_MINSP,2025-04-25,UN_RE,2025-04-25,Fayetteville Road MultiFamily,Phase one of overall development - single fami...,FRANCISCOM,TREYFI,UN_RE,...,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21242,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2014883.724 774772.529),"[(88, units)]"
